In [1]:
!pip install rasterio rasterstats

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 13.6 MB/s eta 0:00:00


In [2]:
import os
from osgeo import gdal, ogr
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Path to the single vector layer (mandal)
mandal_vector_path = "/content/drive/MyDrive/Croptype_Telangana/TS_Mandal_Boundary_632_FINAL.shp"

# Path to the single raster layer
raster_path = "/content/drive/MyDrive/Croptype_Telangana/mosaic_crop_23_24.tif"

In [5]:
# Load the vector layer (mandal)
mandal_gdf = gpd.read_file(mandal_vector_path)

mandal_gdf.head()

,MANDAL_NAM,DISTRICT_N,Shape_Leng,Shape_Area,geometry
0,Abdullapurmet,Rangareddy,98281.218651,2.530733e+08,"POLYGON ((243905.155 1919334.311, 243893.836 1..."
1,Achampet,Nagarkurnool,179345.675426,5.035185e+08,"POLYGON ((277184.657 1830794.289, 277186.255 1..."
2,Adavidevulapally,Nalgonda,60508.049630,1.256258e+08,"POLYGON ((342679.738 1849741.186, 343136.898 1..."
3,Addagudur,Yadadri Bhuvanagiri,57811.266375,1.418131e+08,"POLYGON ((326416.611 1933875.320, 326610.466 1..."
4,Addakal,Mahabubnagar,70669.254384,1.346563e+08,"POLYGON ((171704.385 1835692.421, 171731.846 1..."


In [6]:
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS

# Define the target CRS (EPSG 32644)
target_crs = CRS.from_epsg(32644)

# Open the raster dataset
with rasterio.open(raster_path) as src:
    # Read raster data
    raster_data = src.read(1)

    # Get raster metadata
    transform = src.transform
    nodata = src.nodata

    # Reproject the raster to EPSG 32644
    dst_crs = src.crs if src.crs == target_crs else target_crs
    transform, width, height = calculate_default_transform(src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    # Create a new raster dataset with the WGS 84 CRS
    # with rasterio.open('reprojected_raster.tif', 'w', **kwargs) as dst:
    #     reproject(
    #         source=rasterio.band(src, 1),
    #         destination=rasterio.band(dst, 1),
    #         src_transform=src.transform,
    #         src_crs=src.crs,
    #         dst_transform=transform,
    #         dst_crs=dst_crs,
    #         resampling=Resampling.nearest
    #     )

print("Raster dataset reprojected to WGS 84")

Raster dataset reprojected to WGS 84


In [7]:
# Reproject the GeoDataFrame to EPSG 32644
mandal_gdf = mandal_gdf.to_crs(target_crs)

In [8]:
src.crs, mandal_gdf.crs

(CRS.from_epsg(32644),
 <Projected CRS: EPSG:32644>
 Name: WGS 84 / UTM zone 44N
 Axis Info [cartesian]:
 - [east]: Easting (metre)
 - [north]: Northing (metre)
 Area of Use:
 - undefined
 Coordinate Operation:
 - name: UTM zone 44N
 - method: Transverse Mercator
 Datum: World Geodetic System 1984
 - Ellipsoid: WGS 84
 - Prime Meridian: Greenwich)

In [9]:
raster_data

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [10]:
# unique_categories = [1,2,3,4,5,6]
unique_values = set(raster_data.flatten())
unique_categories = list(unique_values)
unique_categories

[0, 1, 2, 3, 4, 5, 6]

In [11]:
src.res[0] * src.res[1]

100.0

In [15]:
# Initialize a dictionary to store results
results = {
    'District': [],
    'Mandal': [],
}

# Calculate area occupied by each category within each mandal
for category in unique_categories:
    results[f'Area(ha)_Category_{category}'] = []

for index, mandal in mandal_gdf.iterrows():
    geom = mandal.geometry
    stats = zonal_stats(geom, raster_data, categorical=True, nodata=nodata, affine=transform)
    print(stats)
    results['District'].append(mandal['DISTRICT_N'])
    results['Mandal'].append(mandal['MANDAL_NAM'])  # Assuming 'mandal' is the column name for mandal names

    for category in unique_categories:
        if category in list(stats[0].keys()):
            area = stats[0][category] * 0.01 # Converted to hectare (ha)
        else:
            area = 0
        results[f'Area(ha)_Category_{category}'].append(area)

        print(f"Done for {mandal['MANDAL_NAM']}, Category {category}, Area:{area}")

# Convert results dictionary to DataFrame
result_df = pd.DataFrame(results)

/usr/local/lib/python3.10/dist-packages/rasterstats/io.py:328: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


Streaming output truncated to the last 5000 lines.
[{0: 2061460, 1: 94057, 2: 288863, 3: 95602, 4: 246118, 5: 38173}]
Done for Aiza, Category 0, Area:20614.600000000002
Done for Aiza, Category 1, Area:940.57
Done for Aiza, Category 2, Area:2888.63
Done for Aiza, Category 3, Area:956.02
Done for Aiza, Category 4, Area:2461.18
Done for Aiza, Category 5, Area:381.73
Done for Aiza, Category 6, Area:0
[{0: 1218055, 1: 434040, 2: 91782, 4: 51725, 5: 87}]
Done for Akkannapet, Category 0, Area:12180.550000000001
Done for Akkannapet, Category 1, Area:4340.4
Done for Akkannapet, Category 2, Area:917.82
Done for Akkannapet, Category 3, Area:0
Done for Akkannapet, Category 4, Area:517.25
Done for Akkannapet, Category 5, Area:0.87
Done for Akkannapet, Category 6, Area:0
[{0: 1276629, 1: 373085, 2: 10, 4: 3807, 5: 132}]
Done for Alair, Category 0, Area:12766.29
Done for Alair, Category 1, Area:3730.85
Done for Alair, Category 2, Area:0.1
Done for Alair, Category 3, Area:0
Done for Alair, Category 4,

In [16]:
# Display the resulting dataframe
result_df.head()

,District,Mandal,Area(ha)_Category_0,Area(ha)_Category_1,Area(ha)_Category_2,Area(ha)_Category_3,Area(ha)_Category_4,Area(ha)_Category_5,Area(ha)_Category_6
0,Rangareddy,Abdullapurmet,24024.57,1080.78,0.00,0.00,127.82,63.37,10.72
1,Nagarkurnool,Achampet,43559.48,344.80,664.21,620.75,18.65,145.31,4998.99
2,Nalgonda,Adavidevulapally,9753.53,1575.49,0.00,0.00,0.00,803.41,430.16
3,Yadadri Bhuvanagiri,Addagudur,10304.10,3800.48,0.04,0.00,75.16,0.49,0.56
4,Mahabubnagar,Addakal,10772.45,2314.66,27.52,3.76,17.44,0.00,329.35


In [17]:
result_df.to_excel('zonal_stats.xlsx')

In [18]:
result_df[result_df['Mandal']=='Inderavelly']

,District,Mandal,Area(ha)_Category_0,Area(ha)_Category_1,Area(ha)_Category_2,Area(ha)_Category_3,Area(ha)_Category_4,Area(ha)_Category_5,Area(ha)_Category_6
183,Adilabad,Inderavelly,18242.63,0.0,396.39,1723.05,443.41,0.0,0.0


In [19]:
result_df.columns

Index(['District', 'Mandal', 'Area(ha)_Category_0', 'Area(ha)_Category_1',
       'Area(ha)_Category_2', 'Area(ha)_Category_3', 'Area(ha)_Category_4',
       'Area(ha)_Category_5', 'Area(ha)_Category_6'],
      dtype='object')